# Extreme Water level 100yr - RP

Notebook environment to migrate csv files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import standard packages
import os
import pathlib
import sys

import numpy as np
#import geopandas as gpd
 
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import itertools
import json
import copy
from itertools import chain
from shapely import wkb

# Import custom functionality
# sys.path.append('c:/Windows/System32/coclicodata/src')#CLENMAR ADDED

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\01_Hazards"
#gca_data_dir = p_drive.joinpath("11209197-018-global-coastal-atlas","MSc_students","ClenmarRowe")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = pathlib.Path().joinpath(gca_data_dir,"01_Present","Extreme_Sea_Level")
dataset_dir_fut = pathlib.Path().joinpath(gca_data_dir,"02_Future","Extreme_Sea_Level")
dataset_dir_path = dataset_dir.joinpath("Extreme_Sea_Level_original.nc")
CF_dir = dataset_dir.joinpath("CF")  # directory to save output CF check files
CF_dir

WindowsPath('P:/11209197-018-global-coastal-atlas/MSc_students/ClenmarRowe/Data/All_Datasets/Orig_Datasets/01_Hazards/01_Present/Extreme_Sea_Level/CF')

<IPython.core.display.Javascript object>

In [4]:
# # Project paths & files (manual input)
# dataset_dir = gca_data_dir.joinpath(r"All_Datasets")
# dataset_dir_path = dataset_dir.joinpath("EWL_100_original.nc")
# CF_dir = gca_data_dir.joinpath("Example_Coastalrisk_STAC","CFclen")  # directory to save output CF check files

<IPython.core.display.Javascript object>

In [5]:
filename1=dataset_dir.joinpath("ESL_100RP_Baseline.csv")
filename2=dataset_dir_fut.joinpath("ESL_100RP_RCP45-2050.csv")
filename3=dataset_dir_fut.joinpath("ESL_100RP_RCP45-2100.csv")
filename4=dataset_dir_fut.joinpath("ESL_100RP_RCP85-2050.csv")
filename5=dataset_dir_fut.joinpath("ESL_100RP_RCP85-2100.csv")


df1 = pd.read_csv(filename1, index_col=None, header=0)
df2 = pd.read_csv(filename2, index_col=None, header=0)
df3 = pd.read_csv(filename3, index_col=None, header=0)
df4 = pd.read_csv(filename4, index_col=None, header=0)
df5 = pd.read_csv(filename5, index_col=None, header=0)


<IPython.core.display.Javascript object>

In [6]:
df_all = df1.drop(columns=["valueMedian", "value5th", "value95th"])
df_all

,latitude,longitude
0,-16.152930,180.000000
1,-16.216590,179.762811
2,-16.266118,179.523658
3,-16.393551,179.311343
4,-16.445738,179.067616
...,...,...
20860,66.258771,-166.006375
20861,24.471045,54.426563
20862,24.423820,54.448935
20863,37.947217,124.739844


<IPython.core.display.Javascript object>

In [7]:
df_all=df_all.assign(PC_ESL_100=df1['valueMedian'],RCP45_2050_ESL_100=df2['valueMedian'],
                     RCP45_2100_ESL_100=df3['valueMedian'],RCP85_2050_ESL_100=df4['valueMedian'],
                     RCP85_2100_ESL_100=df5['valueMedian'])

df_all


,latitude,longitude,PC_ESL_100,RCP45_2050_ESL_100,RCP45_2100_ESL_100,RCP85_2050_ESL_100,RCP85_2100_ESL_100
0,-16.152930,180.000000,2.333704,2.408321,2.755128,2.613913,3.332276
1,-16.216590,179.762811,2.333704,2.408321,2.755128,2.613913,3.332276
2,-16.266118,179.523658,2.333704,2.408321,2.755128,2.613913,3.332276
3,-16.393551,179.311343,2.299665,2.430962,2.766329,2.583002,3.295478
4,-16.445738,179.067616,2.356555,2.528641,2.870714,2.648645,3.364674
...,...,...,...,...,...,...,...
20860,66.258771,-166.006375,2.259142,2.435123,2.824937,2.643106,3.059973
20861,24.471045,54.426563,1.714927,1.866306,2.181720,1.977713,2.637346
20862,24.423820,54.448935,1.714927,1.866306,2.181720,1.977713,2.637346
20863,37.947217,124.739844,3.556673,3.767853,4.223398,3.867196,4.614292


<IPython.core.display.Javascript object>

In [8]:
ds = xr.Dataset.from_dataframe(df_all)
ds

<xarray.Dataset>
Dimensions:             (index: 20865)
Coordinates:
  * index               (index) int64 0 1 2 3 4 ... 20861 20862 20863 20864
Data variables:
    latitude            (index) float64 -16.15 -16.22 -16.27 ... 37.95 37.92
    longitude           (index) float64 180.0 179.8 179.5 ... 54.45 124.7 124.7
    PC_ESL_100          (index) float64 2.334 2.334 2.334 ... 1.715 3.557 3.557
    RCP45_2050_ESL_100  (index) float64 2.408 2.408 2.408 ... 1.866 3.768 3.768
    RCP45_2100_ESL_100  (index) float64 2.755 2.755 2.755 ... 2.182 4.223 4.223
    RCP85_2050_ESL_100  (index) float64 2.614 2.614 2.614 ... 1.978 3.867 3.867
    RCP85_2100_ESL_100  (index) float64 3.332 3.332 3.332 ... 2.637 4.614 4.614

<IPython.core.display.Javascript object>

In [9]:
# Write the xarray dataset to a netCDF file
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

ds.to_netcdf(str(dataset_dir_path))

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [10]:
# open datasets 
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:             (index: 20865)
Coordinates:
  * index               (index) int64 0 1 2 3 4 ... 20861 20862 20863 20864
Data variables:
    latitude            (index) float64 ...
    longitude           (index) float64 ...
    PC_ESL_100          (index) float64 ...
    RCP45_2050_ESL_100  (index) float64 ...
    RCP45_2100_ESL_100  (index) float64 ...
    RCP85_2050_ESL_100  (index) float64 ...
    RCP85_2100_ESL_100  (index) float64 ...

<IPython.core.display.Javascript object>

In [11]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [12]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\01_Hazards\\01_Present\\Extreme_Sea_Level\\Extreme_Sea_Level_original.nc': {'warnings': '10', 'errors': '0'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [13]:
# open datasets 
ds = xr.open_dataset(dataset_dir_path)

# check original dataset
ds

<xarray.Dataset>
Dimensions:             (index: 20865)
Coordinates:
  * index               (index) int64 0 1 2 3 4 ... 20861 20862 20863 20864
Data variables:
    latitude            (index) float64 ...
    longitude           (index) float64 ...
    PC_ESL_100          (index) float64 ...
    RCP45_2050_ESL_100  (index) float64 ...
    RCP45_2100_ESL_100  (index) float64 ...
    RCP85_2050_ESL_100  (index) float64 ...
    RCP85_2100_ESL_100  (index) float64 ...

<IPython.core.display.Javascript object>

In [14]:
import json

# NetCDF attribute alterations by means of metadata template
metadata_json=dataset_dir.joinpath("ESL_100RP_metadata.json")

f_global = open(metadata_json)
meta_global = json.load(f_global)


for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds.attrs[attr_name] = attr_val

ds.attrs['Conventions'] = "CF-1.8"
ds

<xarray.Dataset>
Dimensions:             (index: 20865)
Coordinates:
  * index               (index) int64 0 1 2 3 4 ... 20861 20862 20863 20864
Data variables:
    latitude            (index) float64 ...
    longitude           (index) float64 ...
    PC_ESL_100          (index) float64 ...
    RCP45_2050_ESL_100  (index) float64 ...
    RCP45_2100_ESL_100  (index) float64 ...
    RCP85_2050_ESL_100  (index) float64 ...
    RCP85_2100_ESL_100  (index) float64 ...
Attributes: (12/20)
    TITLE:               Extreme Sea Level Projections
    TITLE_ABBREVIATION:  ESLP
    DESCRIPTION:         This dataset provides global probabilistic projectio...
    SHORT_DESCRIPTION:   Global probabilistic projections of extreme sea leve...
    INSTITUTION:         Joint Research Centre of the European Commission
    PROVIDERS:           {"name": "Joint Research Centre of the European Comm...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41467-018-04692-w
    LONG_NAME:           EXTREME_SEA_LEVEL_PROJECTIONS
    UNITS:               meters
    COMMENT:             The dataset includes projections for various paramet...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [15]:
ds = ds.rename_dims({"index": "nstations"})

# rename variables, if necessary
ds = ds.rename_vars(
        {"longitude": "lon", "latitude": "lat","index":"stations"} #"index":"nstations", 
  )

# ds=ds.drop(["index"])
# set some data variables to coordinates to avoid duplication of dimensions in later stage

ds = ds.set_coords(["lon", "lat"])
# for var in ds.variables:
#     ds = ds.set_coords(var)

ds

<xarray.Dataset>
Dimensions:             (nstations: 20865)
Coordinates:
  * stations            (nstations) int64 0 1 2 3 4 ... 20861 20862 20863 20864
    lat                 (nstations) float64 ...
    lon                 (nstations) float64 ...
Dimensions without coordinates: nstations
Data variables:
    PC_ESL_100          (nstations) float64 ...
    RCP45_2050_ESL_100  (nstations) float64 ...
    RCP45_2100_ESL_100  (nstations) float64 ...
    RCP85_2050_ESL_100  (nstations) float64 ...
    RCP85_2100_ESL_100  (nstations) float64 ...
Attributes: (12/20)
    TITLE:               Extreme Sea Level Projections
    TITLE_ABBREVIATION:  ESLP
    DESCRIPTION:         This dataset provides global probabilistic projectio...
    SHORT_DESCRIPTION:   Global probabilistic projections of extreme sea leve...
    INSTITUTION:         Joint Research Centre of the European Commission
    PROVIDERS:           {"name": "Joint Research Centre of the European Comm...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41467-018-04692-w
    LONG_NAME:           EXTREME_SEA_LEVEL_PROJECTIONS
    UNITS:               meters
    COMMENT:             The dataset includes projections for various paramet...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [16]:
#Create matrix with Single ESL Variable 
Time_Horizon=[2000,2050,2100] #Columns
#Scenario=["No_Change","RCP_4.5","RCP_8.5"] #Rows
Scenario=[0,4.5,8.5] #Rows

ESL=[[ds["PC_ESL_100"].values,ds["PC_ESL_100"].values,ds["PC_ESL_100"].values],
     [ds["PC_ESL_100"].values,ds["RCP45_2050_ESL_100"].values,ds["RCP45_2100_ESL_100"].values],
     [ds["PC_ESL_100"].values,ds["RCP85_2050_ESL_100"].values,ds["RCP85_2100_ESL_100"].values],
    ]

ESL_array = np.array(ESL)
ESL_array.shape

(3, 3, 20865)

<IPython.core.display.Javascript object>

In [17]:
#Delete all variables, leave only coordinates
keep_vars = []
allvars = list(ds.keys())
delete_vars = list(set(allvars).difference(set(keep_vars)))

ds = ds.drop_vars(delete_vars)
ds

<xarray.Dataset>
Dimensions:   (nstations: 20865)
Coordinates:
  * stations  (nstations) int64 0 1 2 3 4 5 ... 20860 20861 20862 20863 20864
    lat       (nstations) float64 ...
    lon       (nstations) float64 ...
Dimensions without coordinates: nstations
Data variables:
    *empty*
Attributes: (12/20)
    TITLE:               Extreme Sea Level Projections
    TITLE_ABBREVIATION:  ESLP
    DESCRIPTION:         This dataset provides global probabilistic projectio...
    SHORT_DESCRIPTION:   Global probabilistic projections of extreme sea leve...
    INSTITUTION:         Joint Research Centre of the European Commission
    PROVIDERS:           {"name": "Joint Research Centre of the European Comm...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41467-018-04692-w
    LONG_NAME:           EXTREME_SEA_LEVEL_PROJECTIONS
    UNITS:               meters
    COMMENT:             The dataset includes projections for various paramet...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [18]:
ds = ds.expand_dims(dim={"Time_Horizon": Time_Horizon})
ds = ds.expand_dims(dim={"Scenario": Scenario})
ds


<xarray.Dataset>
Dimensions:       (Scenario: 3, Time_Horizon: 3, nstations: 20865)
Coordinates:
  * Scenario      (Scenario) float64 0.0 4.5 8.5
  * Time_Horizon  (Time_Horizon) int32 2000 2050 2100
  * stations      (nstations) int64 0 1 2 3 4 ... 20860 20861 20862 20863 20864
    lat           (nstations) float64 ...
    lon           (nstations) float64 ...
Dimensions without coordinates: nstations
Data variables:
    *empty*
Attributes: (12/20)
    TITLE:               Extreme Sea Level Projections
    TITLE_ABBREVIATION:  ESLP
    DESCRIPTION:         This dataset provides global probabilistic projectio...
    SHORT_DESCRIPTION:   Global probabilistic projections of extreme sea leve...
    INSTITUTION:         Joint Research Centre of the European Commission
    PROVIDERS:           {"name": "Joint Research Centre of the European Comm...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41467-018-04692-w
    LONG_NAME:           EXTREME_SEA_LEVEL_PROJECTIONS
    UNITS:               meters
    COMMENT:             The dataset includes projections for various paramet...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [19]:
# add 3x3x len(station)variable
ds = ds.assign(esl=(["Scenario","Time_Horizon",  "nstations"], ESL_array))
ds

<xarray.Dataset>
Dimensions:       (Scenario: 3, Time_Horizon: 3, nstations: 20865)
Coordinates:
  * Scenario      (Scenario) float64 0.0 4.5 8.5
  * Time_Horizon  (Time_Horizon) int32 2000 2050 2100
  * stations      (nstations) int64 0 1 2 3 4 ... 20860 20861 20862 20863 20864
    lat           (nstations) float64 ...
    lon           (nstations) float64 ...
Dimensions without coordinates: nstations
Data variables:
    esl           (Scenario, Time_Horizon, nstations) float64 2.334 ... 4.614
Attributes: (12/20)
    TITLE:               Extreme Sea Level Projections
    TITLE_ABBREVIATION:  ESLP
    DESCRIPTION:         This dataset provides global probabilistic projectio...
    SHORT_DESCRIPTION:   Global probabilistic projections of extreme sea leve...
    INSTITUTION:         Joint Research Centre of the European Commission
    PROVIDERS:           {"name": "Joint Research Centre of the European Comm...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41467-018-04692-w
    LONG_NAME:           EXTREME_SEA_LEVEL_PROJECTIONS
    UNITS:               meters
    COMMENT:             The dataset includes projections for various paramet...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [20]:
# add or change certain variable / coordinate attributes
### dataset attributes is a dictionary of dictionaries
dataset_attributes = {
   "Scenario": {"long_name": "RCP Scenario", "units": "1"}, # set to 1 if no unit
   "Time_Horizon": {"long_name": "time", "units": "yr"}, # set to 1 if no unit
    "lon": {"standard_name": "longitude", "long_name": "longitude", "units": "degrees_east"},
    "lat": {"standard_name": "latitude", "long_name": "latitude", "units": "degrees_north"},
    "stations": {"long_name": "stations", "units": "1"}, # set to 1 if no unit
    "esl": { "long_name": "Extreme sea level", "units": "m"},
}  # specify custom (CF convention) attributes

 # add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        ds[k].attrs = dataset_attributes[k]
    except:
        continue
ds

<xarray.Dataset>
Dimensions:       (Scenario: 3, Time_Horizon: 3, nstations: 20865)
Coordinates:
  * Scenario      (Scenario) float64 0.0 4.5 8.5
  * Time_Horizon  (Time_Horizon) int32 2000 2050 2100
  * stations      (nstations) int64 0 1 2 3 4 ... 20860 20861 20862 20863 20864
    lat           (nstations) float64 ...
    lon           (nstations) float64 ...
Dimensions without coordinates: nstations
Data variables:
    esl           (Scenario, Time_Horizon, nstations) float64 2.334 ... 4.614
Attributes: (12/20)
    TITLE:               Extreme Sea Level Projections
    TITLE_ABBREVIATION:  ESLP
    DESCRIPTION:         This dataset provides global probabilistic projectio...
    SHORT_DESCRIPTION:   Global probabilistic projections of extreme sea leve...
    INSTITUTION:         Joint Research Centre of the European Commission
    PROVIDERS:           {"name": "Joint Research Centre of the European Comm...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41467-018-04692-w
    LONG_NAME:           EXTREME_SEA_LEVEL_PROJECTIONS
    UNITS:               meters
    COMMENT:             The dataset includes projections for various paramet...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [21]:
# #Change scenario from object to string
# ds["Scenario"] = ds["Scenario"].astype('S')
# ds

<IPython.core.display.Javascript object>

In [22]:
# Write the xarray dataset to a netCDF file
#Compliant netcdf
dataset_dir_path_CF=str(dataset_dir_path).replace("original","final")


ds.to_netcdf(path=dataset_dir_path_CF)

<IPython.core.display.Javascript object>

### Check CF compliancy modified NetCDF files

In [23]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(dataset_dir_path_CF)

# check original dataset
ds

<xarray.Dataset>
Dimensions:       (Scenario: 3, Time_Horizon: 3, nstations: 20865)
Coordinates:
  * Scenario      (Scenario) float64 0.0 4.5 8.5
  * Time_Horizon  (Time_Horizon) int32 2000 2050 2100
    stations      (nstations) int64 ...
    lat           (nstations) float64 ...
    lon           (nstations) float64 ...
Dimensions without coordinates: nstations
Data variables:
    esl           (Scenario, Time_Horizon, nstations) float64 ...
Attributes: (12/20)
    TITLE:               Extreme Sea Level Projections
    TITLE_ABBREVIATION:  ESLP
    DESCRIPTION:         This dataset provides global probabilistic projectio...
    SHORT_DESCRIPTION:   Global probabilistic projections of extreme sea leve...
    INSTITUTION:         Joint Research Centre of the European Commission
    PROVIDERS:           {"name": "Joint Research Centre of the European Comm...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41467-018-04692-w
    LONG_NAME:           EXTREME_SEA_LEVEL_PROJECTIONS
    UNITS:               meters
    COMMENT:             The dataset includes projections for various paramet...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [24]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                 working_dir=CF_dir
                 )

<IPython.core.display.Javascript object>

In [25]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)

{'P:\\11209197-018-global-coastal-atlas\\MSc_students\\ClenmarRowe\\Data\\All_Datasets\\Orig_Datasets\\01_Hazards\\01_Present\\Extreme_Sea_Level\\Extreme_Sea_Level_final.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [26]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(str(dataset_dir_path).replace("original","final").replace(".nc", ".zarr"), mode="w")

<IPython.core.display.Javascript object>